## 🧠 Simulating Neural Connectivity 
The goal is to model neurons in the brain using diffusion limited aggregation.

In [ ]:
%matplotlib osx 
import numpy as np
import matplotlib.pyplot as plt
import math
from matplotlib.animation import FFMpegWriter

In [ ]:
#### DISPLAY FUNCTION 

def display(A):
    #Display the graphics outside of the notebook. 
    #On a PC, use '%matplotlib qt' instead.
    %matplotlib osx 
    plt.imshow(A, cmap="magma");
    plt.axis("off")
    plt.show()
    plt.draw()
    plt.pause(0.01)

In [ ]:
#### VARIABLES 
maxX = 100
maxY = 100
A = np.zeros((maxX, maxY))
midX = A.shape[0] // 2
midY = A.shape[1] // 2

In [ ]:
#### FUNCTIONS 

# initialize a circle at the middle to represent nucleus 
def initializeNucleus(A, radius, centerX, centerY, decay):
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if (i - centerX)**2 + (j - centerY)**2 <= radius**2:
                A[i][j] = 1
            elif (i - centerX)**2 + (j - centerY)**2 <= (radius*1.6)**2:
                A[i][j] = decay

In [ ]:
# grow the neuron's edges with random walks
edgeList = []
def randomWalk(nParticles, decay):
    for i in range(0, nParticles):
        r = np.random.random(); # Random float:  0.0 <= r < 1.0
        x = np.random.randint(maxX - 1)
        y = np.random.randint(maxY - 1)
        if r <= 0.25:      
            x += 1
        elif r <= 0.5 and r > 0.25:  
            x -= 1
        elif r <= 0.75 and r > 0.5:  
            y += 1
        else:         
            y -= 1
        xp, xm, yp, ym = x - 1, x + 1, y - 1, y + 1
        if xm >= maxX or xp <= -1:
            xm = maxX - 1
        if ym >= maxY or yp <= -1:
            ym = maxY - 1
            
        if (A[xm, y] == decay or A[x, yp] == decay or A[xp, y] == decay or A[x, ym] == decay): 
            A[x][y] = decay 
            edgeList.append((x, y))

In [ ]:
# find edge points 
def createEdges(edgeList, decay):
    for (x, y) in edgeList:
        if x + 1 >= maxX or y + 1 >= maxY:
            continue 
        if x - 1 <= 0 or y - 1 <= 0:
            continue 
        conditions = [
            A[x - 1][y + 1] == 0,
            A[x + 1][y + 1] == 0,
            A[x - 1][y - 1] == 0,
            A[x + 1][y - 1] == 0
        ]
        if sum(conditions) >= 3:
            A[x][y] = decay

In [ ]:
# find closest points
def find_closest_matching_points(A, decay):
    rows, cols = A.shape
    min_distance = float('inf')
    groupL = []
    groupR = []
    rad1X, rad1Y = midX + 12, midY + 23
    rad2X, rad2Y = midX - 20, midY - 20
    # first find points that equal decay
    for i in range(rows):
        for j in range(cols):
            value = A[i][j]
            if value != 0 and abs(decay - value) <= 0.1:      # make them closer together so list is most likely not 0 
                dist1 = np.sqrt(((rad1X - i) ** 2) + ((rad1Y - j) ** 2))
                dist2 = np.sqrt(((rad2X - i) ** 2) + ((rad2Y - j) ** 2))
                if dist1 < dist2:
                    groupL.append((i, j))
                else:
                    groupR.append((i, j))
    # then pick a random coordinate from each group
    pick1 = np.random.randint(0, len(groupL))
    pick2 = np.random.randint(0, len(groupR))
    return groupL[pick1], groupR[pick2]

In [ ]:
# connect two neurons using Dijkstra's shortest path algorithm 
import heapq

def dijkstra(matrix, start, end):
    rows, cols = matrix.shape
    distances = np.full_like(matrix, fill_value=np.inf)
    distances[start] = 0
    priority_queue = [(0, start)]
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        if current_node == end:
            break
        for neighbor in neighbors(matrix, current_node):
            new_distance = current_distance + 1  # Assuming a constant cost of 1 for each step
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                heapq.heappush(priority_queue, (new_distance, neighbor))
    return distances

def neighbors(matrix, node):
    x, y = node
    neighbors_list = []
    for i in [-1, 0, 1]:
        for j in [-1, 0, 1]:
            new_x, new_y = x + i, y + j
            if 0 <= new_x < matrix.shape[0] and 0 <= new_y < matrix.shape[1] and matrix[new_x, new_y] != 1:
                neighbors_list.append((new_x, new_y))
    return neighbors_list

In [ ]:
### SETUP 
initializeNucleus(A, 4, midX + 12, midY + 23, 0.8)
initializeNucleus(A, 2, midX - 20, midY - 20, 0.8)

time = 0 
while time < 4:
    randomWalk(1000, 0.8)
    # reinitialize nucleus so that we don't cover it 
    initializeNucleus(A, 4, midX + 12, midY + 23, 0.8)
    initializeNucleus(A, 2, midX - 20, midY - 20, 0.8)
    time += 1
    display(A)
    
    
## ITERATIONS OF EXPANSION
time = 0
decay = 0.8
while time < 200:
    if time != 0 and time % 10 == 0 and decay - 0.04 > 0:
        decay -= 0.04
    createEdges(edgeList, decay)
    edgeList = []        
    randomWalk(1200, decay)
    initializeNucleus(A, 4, midX + 12, midY + 23, 0.8)
    initializeNucleus(A, 2, midX - 20, midY - 20, 0.8)
    time += 1
    display(A)  
print("time iteration 1 complete")
    
time = 0
decay = 0.8
while time < 150:
    if time != 0 and time % 10 == 0 and decay - 0.05 > 0:
        decay -= 0.05
    createEdges(edgeList, decay)
    edgeList = []        
    randomWalk(1200, decay)
    initializeNucleus(A, 4, midX + 12, midY + 23, 0.8)
    initializeNucleus(A, 2, midX - 20, midY - 20, 0.8)
    time += 1
    display(A)    
print("time iteration 2 complete")

## CONNECT THE FURTHEST POINTS TOGETHER AND DRAW PATH
point1, point2 = find_closest_matching_points(A, decay + 0.5)
distances = dijkstra(A, point1, point2)
row, col = point2
action_decay = 0.8
path = []
path.append(point1)
while distances[row, col] != 0:
    min_neighbor = min(neighbors(A, (row, col)), key=lambda x: distances[x])
    row, col = min_neighbor
    path.append((row, col))
path.append(point2)    

for i in range(0, len(path)//2):
    (row, col) = path[i]
    (row2, col2) = path[(-1 * i) - 1]      # when i = 0, path[-1], i = 1, path[-2], i = 2, path[-3]
    A[row][col] = action_decay  
    A[row][col + 1] = action_decay
    A[row][col - 1] = action_decay 
    A[row - 1][col] = action_decay
    A[row + 1][col] = action_decay
    A[row2][col2] = action_decay  
    A[row2][col2 + 1] = action_decay
    A[row2][col2 - 1] = action_decay 
    A[row2 - 1][col2] = action_decay
    A[row2 + 1][col2] = action_decay
    display(A)
    
if len(path) % 2 == 0:
    (midRow, midCol) = path[(len(path) // 2) + 1]
    A[midRow][midCol] = action_decay
    display(A)
    
## SEND ACTION POTENTIAL ACROSS
light = 0 
for (row, col) in path:
    org = A[row][col]
    A[row][col] = light 
    display(A)
    A[row][col] = org